In [1]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import InMemoryByteStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

DATA_DIR = '/mnt/c/Users/David/Documents/Projects/podcast-bot/data'

Single
---

In [2]:

full_text = open("../data/state_of_the_union.txt", "r").read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_text(full_text)
texts

['Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.',
 'Groups of citizens blocking tanks with

Corpus
---

In [3]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import InMemoryByteStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.retrievers.multi_vector import SearchType
import uuid



In [4]:
!ls ../data/12212023_0657/

190.txt  191.txt


In [5]:
loaders = []
folder = os.path.join(DATA_DIR,'12212023_2037')
for filepath in os.listdir(folder):
    filepath = os.path.join(folder, filepath)
    txt = TextLoader(filepath, autodetect_encoding=True)
    loaders.append(txt)
len(loaders)


45

In [6]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)
len(docs)

170

In [7]:
os.environ['OPENAI_API_KEY'] = open('../secrets/openai.txt').read()

In [8]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

retriever.search_type = SearchType.mmr

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [9]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [10]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [11]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [12]:
# Vectorstore alone retrieves the small chunks
retriever.vectorstore.similarity_search("walden by thoreau")

[Document(page_content='hypothetical nature walk, although to Kierkegaard this can be found ANYWHERE...on this nature walk we move away from the temporal realm of human concern into something that more resembles a, sort of, eternal present. When we take a second to stop thinking about our projects in life and return to this type of immediacy...think of the lily that you might COME across on one of these walks.\xa0Consider', metadata={'doc_id': 'c28d0b24-ebef-4836-a462-1a3e3a7dde70', 'source': '/mnt/c/Users/David/Documents/Projects/podcast-bot/data/12212023_2037/pod-160.txt'}),
 Document(page_content='with our relationship to nature. Now once you’re out here…what do you do. Cause it’s not as easy as just SHOWING UP in the woods one day. You don’t just go out into nature and look at the leaves. There’s a special kind of attention you have to pay to nature…a special kind of INTENTION that you have to bring. To Emerson, the FIRST mistake people make when they get AWAY from the city and GO 

In [13]:
# Retriever returns larger chunks
retriever.get_relevant_documents("walden by thoreau")[0].page_content

'first thing maybe to say here is that this is OBVIOUSLY an incredibly dumb way of talking about nature. I mean, Nature walk? Like what you’re going on a pilgrimage to nature now? Hold on, so a bird builds a nest.. and THAT’S nature...but people make buildings and cities and that’s not nature? No, you ARE nature! When you DRIVE to the trail to go on a nature walk there’s no sign that says “Now Entering Nature”. This is an IMPORTANT POINT...because the VALUE... people are getting from what they might call a Nature walk...Kierkegaard would say might have more to do with moving AWAY from this realm up in our heads of human concerns that they’re otherwise serving a life sentence in.\xa0THAT’S the REAL benefit of a nature walk. When they move AWAY from this realm of constant desires and projects...they start to feel CONNECTED to something ELSE...and they’ll describe this something else in a number of ways depending on the language they use: they may say they feel connected to something grea

In [ ]:
from langchain.chains import RetrievalQA
RetrievalQA().from_chain_type(
    llm=llm,
)

In [43]:
def md_search(query: str, search_func=retriever.vectorstore.similarity_search) -> str:
    results = [doc.page_content for doc in search_func(query)]
    output_str = '# Relevant Transcript Snippets\n\n'
    for i,result in enumerate(results):
        output_str += f"## Result {i+1}\n\"{result}\"\n\n"
    return output_str
md_search('walden')

'# Relevant Transcript Snippets\n\n## Result 1\n"show today.So, Walter Benjamin is notorious for being one of the most allusive thinkers of the early 20th century. His work is a combination of influences so varied it’s almost funny. It’s a mix of ideas so seemingly disconnected, the act of connecting them is almost as impressive as the ideas themselves. He’ll go from Kant, then over to Marx, then back to 19th-century German literary criticism,"\n\n## Result 2\n"were in such a state of delirium thinking about how great science and reason were that we ignored one of the biggest breakthroughs in the history of human thought, the call by these thinkers of the Counter-Enlightenment for us to move away from monism and towards what Isaiah Berlin called “pluralism.”There’s a famous essay by Isaiah Berlin titled “The Hedgehog and the Fox.” Now, in this essay he"\n\n## Result 3\n"hypothetical nature walk, although to Kierkegaard this can be found ANYWHERE...on this nature walk we move away from 

---

# Add ReAct agent

In [46]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_community.chat_models import ChatOpenAI


tools = [
    Tool(
        name="Snippets",
        func=md_search,
        description="Retrieve relevant snippets from podcast transcripts",
    ),
    Tool(
        name="Documents",
        func=lambda query: md_search(query,search_func=retriever.get_relevant_documents),
        description="Retrieve full-episode transcripts relevant to a query",
    ),
]

llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview")
# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
react = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [49]:
question = """You are an AI assistant who pretends to be the author of a podcast to answer questions. You are taking on the role of Stephen West, host of the podcast 'Philosophize This!'. You must use transcripts of the podcast to inform your style and replies. You must always use the 'Search' tool.
You will be prompted with a question, create a thoughtful query, search for documents and snippets, and use that information write a reply in the style of the podcast host. 

You MUST respond in the style of the podcast host. You MUST answer in at least 3 sentences. You MUST use a conversational format in your final answer.

Question:
Hey Stephen, hope you're doing well. I was wondering, who's the kind of person that would like Susan Sontag's books?"""
react.run(chat_history="", input=question)



> Entering new AgentExecutor chain...
To answer this question, I need to find information on Susan Sontag's books and the type of content she writes about, which will help me understand the kind of person who would enjoy her work. I should look for episodes where Stephen West discusses Susan Sontag or themes related to her work.

Action: Documents
Action Input: Search for episodes discussing Susan Sontag or themes related to her work.
Observation: # Relevant Transcript Snippets

## Result 1
"So in one of her earlier essays… Susan Sontag talks about a life changing moment she experienced when she was just 12 years old. Now, she described this moment when she wrote about it…as a kind of revelation…in fact, she called it a type of EPIPHANY, that would go on to affect the direction of her thinking AND her work, for DECADES moving into the future. It was July of 1945…she was browsing through a random bookstore in Santa Monica, California…and what she came across was something she would la

"Hey there, thanks for reaching out with such a thoughtful question. You know, Susan Sontag was a master at peeling back the layers of our everyday experiences, revealing the profound and often overlooked complexities beneath. If you're the kind of person who finds themselves pausing to consider the deeper implications of a photograph or a visit to the museum, if you're someone who doesn't just want to remember history but to understand and act upon it, then Sontag's books might just resonate with you. She challenges us to be more than passive consumers of culture; she invites us to be active participants, to question, to critique, and to engage. So if you're someone who's not afraid to confront the uncomfortable or the controversial, to really think about the role of images and narratives in shaping our world, then diving into Sontag's work could be a truly transformative experience. Keep on philosophizing, and let those pages turn!"